## Image Operations

Author: Julian Lißner

For questions and feedback write a mail to: [lissner@mib.uni-stuttgart.de](mailto:lissner@mib.uni-stuttgart.de)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append( 'provided_functions')
import result_check as check
import image_kernels as kernels

sys.path.append( '../submodules')
from general_functions import tic, toc

sys.path.append( 'incomplete_functions' )
import image_operations as operate

- the presented image processing will mostly rely on the convolution operation
- convolution is relevant for computer vision (c.f. convolutional neural networks)
- no algorithm beats the human eye (yet) <br>


consider an image $A \in \mathbb R ^{ M\times N}$ and a kernel of odd size $k\in \mathbb R^{m\times n}$. `//` denotes the floor division.
- convolution shrinks the image by `p_x=n//2`and `p_y = m//2` in each direction
- padding the image with zeros can circumvent undesired effects
- after applying e.g. the convolution, the image needs to be un-padded

<img src='figures/padding.png'>

- the corners need to be filled with 0s as well (for zero padding)
-------------------
**Task:** Implement the functions `zero_padding` and `un_padding` in 'image_operations.py'.y:-y]

In [ ]:
check.padding( operate.zero_padding, operate.un_pad)

- consider the padded image $\underline{\underline A} \in \mathbb R^{(N + n/2)\times (M+ m/2)}$ and the kernel $\underline{\underline k} \in \mathbb R^{n\times m}$
- for all admissible $i$ and $j$, the convolution is computed as
$$C(i,j) = \sum\limits_{o=-\frac n2}^{\frac n2}\,\, \sum\limits_{p=-\frac m2}^{\frac m2} A(i+o,j+p) \cdot k( \frac n2-o, \frac m2-p)$$
- depending on the shape definition of $C$, the resulting $C$ might need to be un padded
- there are multiple ways to correctly implement the convolution using the above formula
---------------------
**Task:** implement the 'sum_convolution' using the above equation in 'image_operations.py'.

In [ ]:
tic( 'sum convolution' )
check.convolution( operate.sum_convolution)
toc( 'sum convolution' )

- convolution is has a computational effort of $\mathcal O((NM)^2)$
- using the FFT, the computational effort can be reduced to $\mathcal O( NM \log(MN) )$
- following the convolution theorem and $\mathcal F(A)$ denoting the FFT of $A$, we get

$$ A \ast k = \mathcal F^{-1} \big( \mathcal F( A) \cdot \mathcal F(k) \big)$$
- pointwise multiplication is required $\blacktriangleright A$ and $k$ need to be of same size
-------------------
**Task:** Implement the 'embed_kernel' function in 'image_operations.py'.
(The result check will give you visual feedback)

In [ ]:
check.embed_kernel( operate.embed_kernel)

- recall the steps for convolution in Fourier space:
    - image padding
    - embed kernel
    - compute convolution
    - un pad image
- note that this convolution is significantly faster than the sum convolution.
-----------------
**Task:** Write the function 'convolution' in 'image_operations.py' performing the convolution based on the convolution theorem.

In [ ]:
tic( 'fft convolution' )
check.convolution( operate.convolution)
toc( 'fft convolution' )

- `sum_convolution` and `convolution` with padding yield the same result
- they differ in numerical precision, see the plot for reference


In [ ]:
images = np.load( 'data/periodic_images.npz')
image = images[ 'arr_0']
kernel = kernels.disc( 37)
fft_conv = operate.convolution( image, kernel)
sum_conv = operate.sum_convolution( image, kernel)

fig, axes = plt.subplots( 1, 3, figsize=(18,6))
axes[0].imshow( fft_conv)
axes[1].imshow( sum_conv)
difference = axes[2].imshow( fft_conv - sum_conv)

plt.colorbar( difference, ax=axes[2])
titles = ['fft convolution', 'sum convolution', 'difference' ]
for num, ax in enumerate(axes):
    ax.axis( 'off')
    ax.set_title( titles[num] )

- if no padding is deployed, they do yield different results
- this is due to an inherent property of the FFT, which induces periodicity
--------------
**Task:** Implement the 'periodic_convolution' function in 'image_operations.py' using the FFT.

In [ ]:
check.periodic_convolution( operate.periodic_convolution)
#check.periodic_convolution( operate.sum_convolution)
## NOTE: you could also implement the periodic convolution as sum convolution using available functions

-------------

## Image Processing via Convolution

- convolution can be used to show the match of a kernel with the image
- the kernel is highlighting how much it matches in the image
- if there is only one activation voxel, the highlight is the kernel
- we will firstly use the convolution to create images by placing the kernel in the image


In [ ]:
resolution = 256
image = np.zeros( 2*[resolution])
specified_activation = [ (25,25), (225, 50), (125, 100), (75, 175)]
for position in specified_activation:
    image[ position] = 1

disc_kernel = kernels.disc(39, kind='ones')
image = operate.convolution( image, disc_kernel)

plt.imshow( image)
plt.axis( 'off')

- different shapes can also be added by simple addition
- this lays two images on top of each other (works here so perfectly because the 'blue' is all 0 )

--------
__Task:__ Add another shape of your liking into the image. Make sure that it is not significantly bigger than the circles.

In [ ]:
extra_kernel = kernels.#TODO kind='ones') #try help( kernels) or dir(kernels)
additional_shapes = np.zeros( image.shape)
specified_activation = [ (210, 230), (150, 200), (65, 75), (25,220) ]
for position in specified_activation:
    additional_shapes[ position] = 1
    
extra_shapes = #TODO
image += #TODO
plt.imshow( image)
plt.axis( 'off')

- use the convolution to detect specified shapes
- applying the convolution shows how much the kernel is represented in the image
- convoluted binary images are generally no longer binary images

-----------
__Task:__ Apply the convolution and review the result. View the peak value in the convoluted image.

In [ ]:
convoluted = operate.convolution( image, disc_kernel)
print( 'sum of the kernel:', disc_kernel.sum() )

plt.imshow( convoluted)
plt.axis( 'off')
plt.colorbar()

- peak activations are given where the image fully matches the kernel
- peak values can be extracted using binary segmentation
- segmentation is defined as: 
$$ A = \begin{cases}  1 & A \geq \theta \\ 0 & else \end{cases}$$

----------
**Task:** Implement the 'segmentation' function in in 'image_operations.py'.

In [ ]:
check.segmentation( operate.segmentation)

- previously implemented functions and applied steps can now be used to detect the circles 
- segmentation is used to find locations where the searched shape is (fully) present
- after a convolution of the found locations with the shape kernel the shape is drawn again
- **Note** if the searched shape is contained within another bigger shape, this simple approach does not work<br>
(e.g. searched shape: small circles, other shape: bigger rectangles)
--------- 
__Task:__ Use the previously introduced methods to find all vertical rectangles in the given image.

In [ ]:
fig, axes = plt.subplots( 2,2, figsize=(12,12))
image = np.load( 'data/detect_shapes.npz')['arr_0']

#searched_shape = kernels.#TODO
#kernel_match = #TODO

#found_shapes = operate.convolution( #TODO, searched_shape) #redraw the locations where the shape was found
#found_shapes = np.minimum( found_shapes, 1)

## the plotting template might help you find the solution, you can plot your owns steps for debugging
axes[0,0].imshow( image)
#axes[0,1].imshow( kernel_match) 
#axes[#TODO....
#axes[1,1].imshow( found_shapes + image) 

titles = [ 'Original image', 'Intermediate step 1', 'Intermediate step 2',  'Detected shapes' ]
for ax in axes.flatten():
    ax.set_title( titles.pop(0))